# 13442 words

In [7]:
import tensorflow as tf
import pickle
from tensorflow.keras import Sequential
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [8]:
def clean_data(text):
    #removing @ tags 
    text=re.sub(r'@[a-zA-z0-9]+\s*',' ',str(text))
    
    #removing urls
    text=re.sub(r'http\S+', '', text)
    text=re.sub(r'www\.\S+', '', text)
    
    #replacing multiple whitesapces by a single
    text = re.sub(r'\s+',' ',text)
    
    # remove all single characters(surrounded by whitespace)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # remove all single characters except i and I (surrounded by whitespace)
    # text = re.sub(r'\s+(?![iI])[a-zA-Z]\s+', ' ', text)
    
    
    # Converting to Lowercase 
    text = text.lower()
    
    # Lemmatization- splits into list of words ['The', 'quick', ....]
    text = text.split()

    lemma = WordNetLemmatizer()
    text = [lemma.lemmatize(word) for word in text]
    text = ' '.join(text)
    
    words = nltk.word_tokenize(text)
    
    # Get the list of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords from the text
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Recreate the text without stopwords
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

In [9]:
with open('countvectorizer.pkl', 'rb') as f:
    count_vectorizer = pickle.load(f)

/Users/himanshupradhan/coding/Projects/ML/recomend-song-from-text-image/venv/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
sentiment_mapping={
    '0':'sadness',
    '1':'joy',
    '2':'love',
    '3':'anger',
    '4':'fear',
    '5':['enthusiasm','surprise'],
}

In [11]:
#setting up shape there must be a simpler way
X=['makes feel wonderful']

In [12]:
X=count_vectorizer.transform(X).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0]])

In [13]:
_,m=X.shape

In [14]:
opt=tf.keras.optimizers.Adam()
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
batch_size=1000

In [15]:
model=Sequential(
      [tf.keras.Input(shape=(m,)),
      tf.keras.layers.Dense(units=3000,activation='relu'),
      tf.keras.layers.Dense(units=600,activation='relu'),
      tf.keras.layers.Dense(units=400,activation='relu'),
      tf.keras.layers.Dense(units=260,activation='relu'),
      tf.keras.layers.Dense(units=30,activation='relu'),
      tf.keras.layers.Dense(units=6,activation='linear')])
    
model.compile(loss=loss,optimizer=opt,metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3000)              40329000  
                                                                 
 dense_1 (Dense)             (None, 600)               1800600   
                                                                 
 dense_2 (Dense)             (None, 400)               240400    
                                                                 
 dense_3 (Dense)             (None, 260)               104260    
                                                                 
 dense_4 (Dense)             (None, 30)                7830      
                                                                 
 dense_5 (Dense)             (None, 6)                 186       
                                                                 
Total params: 42482276 (162.06 MB)
Trainable params: 424

In [16]:
model.load_weights('model.h5')

In [21]:
X=[
    "why are you like this??",
]
X=[clean_data(x) for x in X]
print(X)
X=count_vectorizer.transform(X).toarray()
X

['like ? ?']


array([[0, 0, 0, ..., 0, 0, 0]])

In [22]:
y=model.predict(X)
sentiment_mapping[str(np.argmax(y))]


1/1 [==============================] - 0s 19ms/step


'joy'

In [19]:
y_sm=tf.nn.softmax(y)
y_sm

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[4.8960590e-07, 5.6916146e-05, 1.7696588e-05, 8.6705347e-07,
        4.6127807e-05, 9.9987793e-01]], dtype=float32)>

In [20]:
pred=np.argmax(y_sm)
pred

5